In [9]:
pip install -q pyathena

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.0.7 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd


session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()

# use default bucket
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-075039479415


load the dataset

In [6]:
df = pd.read_csv("data/Churn_Modelling.csv")
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Upload the data to the S3 bucket

In [7]:
data_path = "data/Churn_Modelling.csv"
s3_path = "final_project/Churn_Modelling.csv"
s3.upload_file(Filename=data_path, Bucket=bucket, Key=s3_path)

# Create Athena Database

In [8]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

database_name = "customer_data"
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)

In [9]:
print(statement)

CREATE DATABASE IF NOT EXISTS customer_data


In [10]:
# create the database
pd.read_sql(statement, conn)

/tmp/ipykernel_238/914221194.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [11]:
# check if the database exists
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_238/2064046450.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,customer_data
1,default
2,dsoaws
3,hw2database
4,sagemaker_featurestore


# Register data with Athena

In [16]:
table_name = "churn_prediction"
s3_private_path_tsv = "s3://{}/final_project".format(bucket)

# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    RowNumber INT,
    CustomerID int,
    Surname string,
    CreditScore int,
    Geography string,
    Gender string,
    Age int,
    Tenure int,
    Balance float,
    NumOfProducts int,
    HasCrCard int,
    IsActiveMember int,
    EstimatedSalary float,
    Exited int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name, s3_private_path_tsv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS customer_data.churn_prediction(
    RowNumber INT,
    CustomerID int,
    Surname string,
    CreditScore int,
    Geography string,
    Gender string,
    Age int,
    Tenure int,
    Balance float,
    NumOfProducts int,
    HasCrCard int,
    IsActiveMember int,
    EstimatedSalary float,
    Exited int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-075039479415/final_project'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [17]:
pd.read_sql(statement, conn)

/tmp/ipykernel_238/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [18]:
# verify if the table is created successfully
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_238/3122712337.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,churn_prediction


In [40]:
# Run A Sample Query

customer_id = 15634602

statement = """SELECT * FROM {}.{}
    WHERE customerid = {} LIMIT 100""".format(
    database_name, table_name, customer_id
)

print(statement)

SELECT * FROM customer_data.churn_prediction
    WHERE customerid = 15634602 LIMIT 100


In [41]:
df_result = pd.read_sql(statement, conn)
df_result

/tmp/ipykernel_238/646157818.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(statement, conn)


,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1
